## Теперь о монетизации - Sleepy работает по подписке: 990 рублей в месяц. Первые две недели бесплатно, затем клиент видит пейвол (от «pay wall» - как бы стена, которая тебя не пускает дальше, пока ты не оплатишь) и, если привязывает карту, продолжает пользоваться подпиской, а деньги списываются автоматически.

## Если говорить о привлечении, то это молодой венчурный стартап, поэтому привлечение одного пользователя (даже если он не заплатит) довольно дорогое - 500 рублей.

## Данных у вас немного - вам дан датасет всего из двух полей:


## user - номер пользователя в системе

## dt - дата его захода в приложение

## Данные устроены таким образом, что фиксируется только один заход пользователя за день - одинаковых пар "пользователь - дата" быть не может. 
## Кроме того, фиксируются только заходы пользователей на триальной версии и тех, кто прошел пейвол. 
## Если человек открыл приложение после 14 дня и не привязал карту - такой заход не фиксируется.

In [82]:
import pandas as pd
import math

In [103]:
# n day retention


# WIN
#users_sleepy = pd.read_csv(r'C:\Users\Incognitus\Downloads\entries.csv', sep=';') 
# MAC
users_sleepy = pd.read_csv(r'/Users/vladislavlipkin/Downloads/entries.csv', sep=';') 

In [104]:
users_sleepy.head(5)

,user,dt,Unnamed: 2
0,0,12.11.2023,NaN
1,0,13.11.2023,NaN
2,0,14.11.2023,NaN
3,0,16.11.2023,NaN
4,0,17.11.2023,NaN


In [127]:
# Берем нужные столбцы из исходного датафрейма
users_filter = users_sleepy.loc[:, ['user', 'dt']]

# Преобразуем столбец dt в формат datetime
users_filter['dt'] = pd.to_datetime(users_filter['dt'], format='%d.%m.%Y')

# Определяем дату первого визита (регистрации) каждого пользователя
users_filter['dt_reg'] = users_filter.groupby('user')['dt'].transform('min')

# Считаем разницу в днях между визитом и регистрацией
users_filter['diff'] = (users_filter['dt'] - users_filter['dt_reg']).dt.days

# Считаем общее количество уникальных пользователей
all_users = users_filter['user'].nunique()

# Вычисляем retention: доля уникальных пользователей в каждом diff
users_filter['retention'] = users_filter.groupby('diff')['user'].transform('nunique') / all_users * 100

# Сводная таблица: и retention, и количество уникальных пользователей
pivot = users_filter.pivot_table(
    index='diff',
    values='user',
    aggfunc='nunique'   # считаем уникальных пользователей
).rename(columns={'user': 'unique_users'}).reset_index()

# Добавляем колонку retention в тот же pivot
pivot['retention'] = pivot['unique_users'] / all_users * 100

# Создаем полный диапазон diff от min до max (чтобы не было пропусков)
full_range = pd.DataFrame({'diff': range(pivot['diff'].min(), pivot['diff'].max() + 1)})

# Объединяем полный диапазон с pivot (left join)
pivot_full = full_range.merge(pivot, on='diff', how='left')

# Просматриваем первые строки итоговой таблицы
pivot_full.head(20)

,diff,unique_users,retention
0,0,2000.0,100.00
1,1,1039.0,51.95
2,2,1052.0,52.60
3,3,1055.0,52.75
4,4,1024.0,51.20
5,5,968.0,48.40
6,6,984.0,49.20
7,7,927.0,46.35
8,8,954.0,47.70
9,9,958.0,47.90


In [149]:
# count users paying
cpu = users_filter.query('diff >= 14')['user'].nunique()
print(f'количество людей купивших подписку : {cpu}')
print()
price_all_users = all_users * 500
net_profit = cpu * 990 - price_all_users
print(f'стоимость привлечения за всех пользователей {price_all_users}')
print()
print(f'выручка за все проданные подписки: {cpu * 990}')
print()
print(f'чистая прибыль с учетом затратов: {net_profit}')

количество людей купивших подписку : 1981

стоимость привлечения за всех пользователей 1000000

выручка за все проданные подписки: 1961190

чистая прибыль с учетом затратов: 961190


In [167]:
pivot_full['retention'].sum() / 100

np.float64(26.162500000000005)

In [162]:
uniq_users = users_filter.query('14 <= diff <= 16')['user'].nunique()
print(f'количество уникальных пользователей: {uniq_users}')
print(f'выручка: {uniq_users * 990}')

количество уникальных пользователей: 1076
выручка: 1065240


In [139]:
pivot_full.query('diff >= 14').sort_values('retention', ascending=False)

,diff,unique_users,retention
16,16,518.0,25.90
14,14,500.0,25.00
15,15,484.0,24.20
18,18,478.0,23.90
17,17,465.0,23.25
...,...,...,...
327,327,2.0,0.10
344,344,2.0,0.10
341,341,2.0,0.10
364,364,2.0,0.10


In [138]:
users_filter.query('diff >= 14')['user'].nunique() - 500

1481

In [128]:
pivot_full.query('diff >= 14')

,diff,unique_users,retention
14,14,500.0,25.00
15,15,484.0,24.20
16,16,518.0,25.90
17,17,465.0,23.25
18,18,478.0,23.90
...,...,...,...
360,360,6.0,0.30
361,361,5.0,0.25
362,362,3.0,0.15
363,363,9.0,0.45
